# Generating ML XS, ACE Files, Benchmark Inputs and Serpent Scripts

In [100]:
import pandas as pd
import os
import logging

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
pd.options.mode.chained_assignment = None  # default='warn'

import sys
# This allows us to import the nucml utilities
sys.path.append("..")


import nucml.datasets as nuc_data
import nucml.ace.data_utilities as ace_utils
import nucml.model.utilities as model_utils

In [101]:
figure_dir = "Figures/"

In [102]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [103]:
import importlib
importlib.reload(nuc_data)
importlib.reload(ace_utils)
importlib.reload(model_utils)
print("Finish re-loading scripts.")

Finish re-loading scripts.


In [104]:
import nucml.exfor.data_utilities as exfor_utils
import nucml.general_utilities as gen_utils
import nucml.config as config
importlib.reload(config)
importlib.reload(gen_utils)
importlib.reload(exfor_utils)

<module 'nucml.exfor.data_utilities' from '..\\nucml\\exfor\\data_utilities.py'>

## Loading Datasets Dataset

In [6]:
# LOADING DATASET
df_b0, _, _, _, _, to_scale_b0, _ = nuc_data.load_exfor(pedro=True, basic=0, normalize=False)
df_b1, _, _, _, _, to_scale_b1, _ = nuc_data.load_exfor(pedro=True, basic=1, normalize=False)
df_b2, _, _, _, _, to_scale_b2, _ = nuc_data.load_exfor(pedro=True, basic=2, normalize=False)
df_b3, _, _, _, _, to_scale_b3, _ = nuc_data.load_exfor(pedro=True, basic=3, normalize=False)
df_b4, _, _, _, _, to_scale_b4, _ = nuc_data.load_exfor(pedro=True, basic=4, normalize=False)

## Decision Tree

In [9]:
dt_ml_ace_dir_b0 = "ml/DT_B0/"
dt_ml_ace_dir_b1 = "ml/DT_B1/"
dt_ml_ace_dir_b2 = "ml/DT_B2/"
dt_ml_ace_dir_b3 = "ml/DT_B3/"
dt_ml_ace_dir_b4 = "ml/DT_B4/"

In [39]:
results_b0 = pd.read_csv("../ML_EXFOR_neutrons/2_DT/dt_resultsB0.csv").sort_values(by="max_depth")
results_b1 = pd.read_csv("../ML_EXFOR_neutrons/2_DT/dt_resultsB1.csv").sort_values(by="max_depth")
results_b2 = pd.read_csv("../ML_EXFOR_neutrons/2_DT/dt_resultsB2.csv").sort_values(by="max_depth")
results_b3 = pd.read_csv("../ML_EXFOR_neutrons/2_DT/dt_resultsB3.csv").sort_values(by="max_depth")
results_b4 = pd.read_csv("../ML_EXFOR_neutrons/2_DT/dt_resultsB4.csv").sort_values(by="max_depth")

results_b0 = results_b0[results_b0.normalizer == "none"]

In [54]:
# filter down to just three for testing purposes:
dt_results = model_utils.get_best_models_df(results_b1).iloc[[0]]

# BENCHMARK_NAME = "U233_MET_FAST_001"
# BENCHMARK_NAME_2 = "U233_MET_FAST_002_001"
BENCHMARK_NAME = "PU_MET_FAST_005"
ace_utils.generate_bench_ml_xs(df_b1, dt_results, BENCHMARK_NAME, to_scale_b1, "TESTING", reset=True)

In [52]:
BENCHMARK_NAME = "U233_MET_FAST_001"

In [ ]:
ace_utils.generate_bench_ml_xs(df_b0, results_b0, BENCHMARK_NAME, to_scale_b0, dt_ml_ace_dir_b0, reset=True)
ace_utils.generate_bench_ml_xs(df_b1, results_b1, BENCHMARK_NAME, to_scale_b1, dt_ml_ace_dir_b1, reset=True)
ace_utils.generate_bench_ml_xs(df_b2, results_b2, BENCHMARK_NAME, to_scale_b2, dt_ml_ace_dir_b2, reset=True)
ace_utils.generate_bench_ml_xs(df_b3, results_b3, BENCHMARK_NAME, to_scale_b3, dt_ml_ace_dir_b3, reset=True)
ace_utils.generate_bench_ml_xs(df_b4, results_b4, BENCHMARK_NAME, to_scale_b4, dt_ml_ace_dir_b4, reset=True)

In [53]:
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b0, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b1, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b2, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b3, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b4, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

In [48]:
BENCHMARK_NAME = "U233_MET_FAST_002_001"

In [45]:
ace_utils.generate_bench_ml_xs(df_b0, results_b0, BENCHMARK_NAME, to_scale_b0, dt_ml_ace_dir_b0, reset=True)
ace_utils.generate_bench_ml_xs(df_b1, results_b1, BENCHMARK_NAME, to_scale_b1, dt_ml_ace_dir_b1, reset=True)
ace_utils.generate_bench_ml_xs(df_b2, results_b2, BENCHMARK_NAME, to_scale_b2, dt_ml_ace_dir_b2, reset=True)
ace_utils.generate_bench_ml_xs(df_b3, results_b3, BENCHMARK_NAME, to_scale_b3, dt_ml_ace_dir_b3, reset=True)
ace_utils.generate_bench_ml_xs(df_b4, results_b4, BENCHMARK_NAME, to_scale_b4, dt_ml_ace_dir_b4, reset=True)

In [49]:
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b0, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b1, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b2, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b3, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b4, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

In [50]:
BENCHMARK_NAME = "U233_MET_FAST_002_002"

In [46]:
ace_utils.generate_bench_ml_xs(df_b0, results_b0, BENCHMARK_NAME, to_scale_b0, dt_ml_ace_dir_b0, reset=True)
ace_utils.generate_bench_ml_xs(df_b1, results_b1, BENCHMARK_NAME, to_scale_b1, dt_ml_ace_dir_b1, reset=True)
ace_utils.generate_bench_ml_xs(df_b2, results_b2, BENCHMARK_NAME, to_scale_b2, dt_ml_ace_dir_b2, reset=True)
ace_utils.generate_bench_ml_xs(df_b3, results_b3, BENCHMARK_NAME, to_scale_b3, dt_ml_ace_dir_b3, reset=True)
ace_utils.generate_bench_ml_xs(df_b4, results_b4, BENCHMARK_NAME, to_scale_b4, dt_ml_ace_dir_b4, reset=True)

In [51]:
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b0, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b1, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b2, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b3, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b4, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

In [ ]:
BENCHMARK_NAME = "PU_MET_FAST_005"

## K-Nearest-Neighbors

In [112]:
knn_ml_ace_dir_b0 = "ml/KNN_B0/"
knn_ml_ace_dir_b1 = "ml/KNN_B1/"
knn_ml_ace_dir_b2 = "ml/KNN_B2/"
knn_ml_ace_dir_b3 = "ml/KNN_B3/"
knn_ml_ace_dir_b4 = "ml/KNN_B4/"

In [113]:
# results_b0 = pd.read_csv("../ML_EXFOR_neutrons/1_KNN/knn_results_B0.csv").sort_values(by="id")
results_b1 = pd.read_csv("../ML_EXFOR_neutrons/1_KNN/knn_results_B1.csv").sort_values(by="id")
results_b2 = pd.read_csv("../ML_EXFOR_neutrons/1_KNN/knn_results_B2.csv").sort_values(by="id")
results_b3 = pd.read_csv("../ML_EXFOR_neutrons/1_KNN/knn_results_B3.csv").sort_values(by="id")
results_b4 = pd.read_csv("../ML_EXFOR_neutrons/1_KNN/knn_results_B4.csv").sort_values(by="id")

# results_b0["scale_energy"] = results_b0.run_name.apply(lambda x: True if "v2" in x else False)
# results_b0["Model"] = results_b0.model_path.apply(lambda x: os.path.basename(os.path.dirname(x)))

# results_b0 = results_b0[results_b0.normalizer == "minmax"]
# results_b0 = results_b0[results_b0.scale_energy == True]
# results_b0 = results_b0[results_b0.distance_metric == 'manhattan']

In [114]:
BENCHMARK_NAME = "U233_MET_FAST_001"

In [69]:
# ace_utils.generate_bench_ml_xs(df_b0, results_b0, BENCHMARK_NAME, to_scale_b0, knn_ml_ace_dir_b0, reset=True)
ace_utils.generate_bench_ml_xs(df_b1, results_b1, BENCHMARK_NAME, to_scale_b1, knn_ml_ace_dir_b1, reset=True)
ace_utils.generate_bench_ml_xs(df_b2, results_b2, BENCHMARK_NAME, to_scale_b2, knn_ml_ace_dir_b2, reset=True)
ace_utils.generate_bench_ml_xs(df_b3, results_b3, BENCHMARK_NAME, to_scale_b3, knn_ml_ace_dir_b3, reset=True)
ace_utils.generate_bench_ml_xs(df_b4, results_b4, BENCHMARK_NAME, to_scale_b4, knn_ml_ace_dir_b4, reset=True)

In [71]:
# ace_utils.generate_serpent_bash(knn_ml_ace_dir_b0, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(knn_ml_ace_dir_b1, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(knn_ml_ace_dir_b2, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(knn_ml_ace_dir_b3, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(knn_ml_ace_dir_b4, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

In [115]:
BENCHMARK_NAME = "U233_MET_FAST_002_001"

In [116]:
# ace_utils.generate_bench_ml_xs(df_b0, results_b0, BENCHMARK_NAME, to_scale_b0, knn_ml_ace_dir_b0, reset=True)
ace_utils.generate_bench_ml_xs(df_b1, results_b1[3:], BENCHMARK_NAME, to_scale_b1, knn_ml_ace_dir_b1, reset=True)
# ace_utils.generate_bench_ml_xs(df_b2, results_b2, BENCHMARK_NAME, to_scale_b2, knn_ml_ace_dir_b2, reset=True)
# ace_utils.generate_bench_ml_xs(df_b3, results_b3, BENCHMARK_NAME, to_scale_b3, knn_ml_ace_dir_b3, reset=True)
# ace_utils.generate_bench_ml_xs(df_b4, results_b4, BENCHMARK_NAME, to_scale_b4, knn_ml_ace_dir_b4, reset=True)

In [117]:
# ace_utils.generate_serpent_bash(knn_ml_ace_dir_b0, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(knn_ml_ace_dir_b1, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(knn_ml_ace_dir_b2, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(knn_ml_ace_dir_b3, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(knn_ml_ace_dir_b4, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

In [118]:
BENCHMARK_NAME = "U233_MET_FAST_002_002"

In [119]:
# ace_utils.generate_bench_ml_xs(df_b0, results_b0, BENCHMARK_NAME, to_scale_b0, knn_ml_ace_dir_b0, reset=True)
ace_utils.generate_bench_ml_xs(df_b1, results_b1, BENCHMARK_NAME, to_scale_b1, knn_ml_ace_dir_b1, reset=True)
# ace_utils.generate_bench_ml_xs(df_b2, results_b2, BENCHMARK_NAME, to_scale_b2, knn_ml_ace_dir_b2, reset=True)
# ace_utils.generate_bench_ml_xs(df_b3, results_b3, BENCHMARK_NAME, to_scale_b3, knn_ml_ace_dir_b3, reset=True)
# ace_utils.generate_bench_ml_xs(df_b4, results_b4, BENCHMARK_NAME, to_scale_b4, knn_ml_ace_dir_b4, reset=True)

In [120]:
# ace_utils.generate_serpent_bash(knn_ml_ace_dir_b0, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(knn_ml_ace_dir_b1, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(knn_ml_ace_dir_b2, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(knn_ml_ace_dir_b3, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(knn_ml_ace_dir_b4, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

## XGBoost

In [121]:
xgb_ml_ace_dir_b0 = "ml/XGB_B0/"
xgb_ml_ace_dir_b1 = "ml/XGB_B1/"
xgb_ml_ace_dir_b2 = "ml/XGB_B2/"
xgb_ml_ace_dir_b3 = "ml/XGB_B3/"
xgb_ml_ace_dir_b4 = "ml/XGB_B4/"

In [122]:
results_b0 = pd.read_csv("../ML_EXFOR_neutrons/3_XGB/xgb_resultsB0.csv")
results_b1 = pd.read_csv("../ML_EXFOR_neutrons/3_XGB/xgb_resultsB1.csv")
results_b2 = pd.read_csv("../ML_EXFOR_neutrons/3_XGB/xgb_resultsB2.csv")
results_b3 = pd.read_csv("../ML_EXFOR_neutrons/3_XGB/xgb_resultsB3.csv")
results_b4 = pd.read_csv("../ML_EXFOR_neutrons/3_XGB/xgb_resultsB4.csv")

In [123]:
BENCHMARK_NAME = "U233_MET_FAST_001"

In [124]:
ace_utils.generate_bench_ml_xs(df_b0, results_b0, BENCHMARK_NAME, to_scale_b0, xgb_ml_ace_dir_b0, reset=True)
# ace_utils.generate_bench_ml_xs(df_b1, results_b1, BENCHMARK_NAME, to_scale_b1, xgb_ml_ace_dir_b1, reset=True)
ace_utils.generate_bench_ml_xs(df_b2, results_b2, BENCHMARK_NAME, to_scale_b2, xgb_ml_ace_dir_b2, reset=True)
ace_utils.generate_bench_ml_xs(df_b3, results_b3, BENCHMARK_NAME, to_scale_b3, xgb_ml_ace_dir_b3, reset=True)
ace_utils.generate_bench_ml_xs(df_b4, results_b4, BENCHMARK_NAME, to_scale_b4, xgb_ml_ace_dir_b4, reset=True)

In [125]:
ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b0, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b1, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b2, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b3, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b4, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

In [126]:
BENCHMARK_NAME = "U233_MET_FAST_002_001"

In [127]:
# ace_utils.generate_bench_ml_xs(df_b0, results_b0, BENCHMARK_NAME, to_scale_b0, xgb_ml_ace_dir_b0, reset=True)
ace_utils.generate_bench_ml_xs(df_b1, results_b1, BENCHMARK_NAME, to_scale_b1, xgb_ml_ace_dir_b1, reset=True)
# ace_utils.generate_bench_ml_xs(df_b2, results_b2, BENCHMARK_NAME, to_scale_b2, xgb_ml_ace_dir_b2, reset=True)
# ace_utils.generate_bench_ml_xs(df_b3, results_b3, BENCHMARK_NAME, to_scale_b3, xgb_ml_ace_dir_b3, reset=True)
# ace_utils.generate_bench_ml_xs(df_b4, results_b4, BENCHMARK_NAME, to_scale_b4, xgb_ml_ace_dir_b4, reset=True)

In [128]:
# ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b0, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b1, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b2, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b3, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b4, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

In [129]:
BENCHMARK_NAME = "U233_MET_FAST_002_002"

In [130]:
# ace_utils.generate_bench_ml_xs(df_b0, results_b0, BENCHMARK_NAME, to_scale_b0, xgb_ml_ace_dir_b0, reset=True)
ace_utils.generate_bench_ml_xs(df_b1, results_b1, BENCHMARK_NAME, to_scale_b1, xgb_ml_ace_dir_b1, reset=True)
# ace_utils.generate_bench_ml_xs(df_b2, results_b2, BENCHMARK_NAME, to_scale_b2, xgb_ml_ace_dir_b2, reset=True)
# ace_utils.generate_bench_ml_xs(df_b3, results_b3, BENCHMARK_NAME, to_scale_b3, xgb_ml_ace_dir_b3, reset=True)
# ace_utils.generate_bench_ml_xs(df_b4, results_b4, BENCHMARK_NAME, to_scale_b4, xgb_ml_ace_dir_b4, reset=True)

In [131]:
# ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b0, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b1, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b2, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b3, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
# ace_utils.generate_serpent_bash(xgb_ml_ace_dir_b4, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

In [11]:
# all_serpent_files = []

# for root, _, files in os.walk("ml/DT_B0"):
#     for file in files:
#         if "U233_MET_FAST_001_001" in root:
#             all_serpent_files.append(os.path.abspath(os.path.join(root, file)))